<a href="https://colab.research.google.com/github/amitadhainje/LearningBERT/blob/master/Transformer_In_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRANSFORMERS**<br />


**STEP - 1 - IMPORTING REQUIRED LIBRARIES**

In [0]:
import numpy as np
import math
import re
import time
from google.colab import drive

try:
    %tensorflow_version 2.x
except:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

**STEP - 2 - IMPORT THE DATA FROM GOOGLE DRIVE**

In [2]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls -ltr /content/drive/'My Drive'/'Colab Notebooks'/NLP/nonbreaking_prefix.fr

-rw------- 1 root root 121 May  1 09:05 '/content/drive/My Drive/Colab Notebooks/NLP/nonbreaking_prefix.fr'


In [0]:
with open("/content/drive/My Drive/Colab Notebooks/NLP/nonbreaking_prefix.en", mode='r', encoding='utf-8') as f:
    non_breaking_prefix_en = f.read()
with open("/content/drive/My Drive/Colab Notebooks/NLP/nonbreaking_prefix.fr", mode='r', encoding='utf-8') as f:
    non_breaking_prefix_fr = f.read()

In [5]:
print ("Following is the input file in english language - ")
with open("/content/drive/My Drive/Colab Notebooks/NLP/europarl-v7.fr-en.en", mode='r', encoding='utf-8') as f:
    europarl_en = f.read()
europarl_en[0:100]

Following is the input file in english language - 


'Resumption of the session\nI declare resumed the session of the European Parliament adjourned on Frid'

In [6]:
print ("Following is the input file in french language - ")
with open("/content/drive/My Drive/Colab Notebooks/NLP/europarl-v7.fr-en.fr", mode='r', encoding='utf-8') as f:
    europarl_fr = f.read()
europarl_fr[0:100]

Following is the input file in french language - 


'Reprise de la session\nJe déclare reprise la session du Parlement européen qui avait été interrompue '

The above files named "europarl-v7.fr-en.fr"  and "europarl-v7.fr-en.en" are files containing french and english sentences respectively. We need to convert it into a list of sentences. Hence we will be using the "nonbreaking" prefix files for the same. The following step of "Cleaning the data" will convert the text in the english and french files into a list of sentences.


**STEP - 3 - CLEANING DATA**

In [0]:
non_breaking_prefix_en = non_breaking_prefix_en.split("\n")
non_breaking_prefix_en = [' ' + pref + '.' for pref in non_breaking_prefix_en]
non_breaking_prefix_fr = non_breaking_prefix_fr.split("\n")
non_breaking_prefix_fr = [' ' + pref + '.' for pref in non_breaking_prefix_fr]

In [8]:
non_breaking_prefix_en

[' a.',
 ' b.',
 ' c.',
 ' d.',
 ' e.',
 ' f.',
 ' g.',
 ' h.',
 ' i.',
 ' j.',
 ' k.',
 ' l.',
 ' m.',
 ' n.',
 ' o.',
 ' p.',
 ' q.',
 ' r.',
 ' s.',
 ' t.',
 ' u.',
 ' v.',
 ' w.',
 ' x.',
 ' y.',
 ' z.',
 ' messrs.',
 ' mlle.',
 ' mme.',
 ' mr.',
 ' mrs.',
 ' ms.',
 ' ph.',
 ' prof.',
 ' sr.',
 ' st.',
 ' a.m.',
 ' p.m.',
 ' vs.',
 ' i.e.',
 ' e.g.']

In [0]:
corpus_en = europarl_en
# Add $$$ after non ending sentence points
for prefix in non_breaking_prefix_en:
    corpus_en = corpus_en.replace(prefix, prefix + '$$$')
corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_en)
# Remove $$$ markers
corpus_en = re.sub(r".\$\$\$", '', corpus_en)
# Clear multiple spaces
corpus_en = re.sub(r"  +", " ", corpus_en)
corpus_en = corpus_en.split('\n')

corpus_fr = europarl_fr
for prefix in non_breaking_prefix_fr:
    corpus_fr = corpus_fr.replace(prefix, prefix + '$$$')
corpus_fr = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_fr)
corpus_fr = re.sub(r".\$\$\$", '', corpus_fr)
corpus_fr = re.sub(r"  +", " ", corpus_fr)
corpus_fr = corpus_fr.split('\n')

In [12]:
print ("Few english sentences are as follows - ")
for x in corpus_en[0:5]:
    print (x)
print ("\n")
print ("Few french sentences are as follows - ")
for x in corpus_fr[0:5]:
    print (x)
print ("\n")

Few english sentences are as follows - 
Resumption of the session
I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
You have requested a debate on this subject in the course of the next few days, during this part-session.
In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.


Few french sentences are as follows - 
Reprise de la session
Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle 

**STEP - 4 - TOKENIZATION OF DATA**

In [0]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(corpus_en, target_vocab_size=2**9)
tokenizer_fr = tfds.features.text.SubwordTextEncoder.build_from_corpus(corpus_fr, target_vocab_size=2**9)

We are going to add 2 tokens to our sentences. One at the start of the sentence and another at the end of the sentences. We are going to use this for our model

In [0]:
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2
VOCAB_SIZE_FR = tokenizer_fr.vocab_size + 2

In [19]:
inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
          for sentence in corpus_en]
outputs = [[VOCAB_SIZE_FR-2] + tokenizer_fr.encode(sentence) + [VOCAB_SIZE_FR-1]
           for sentence in corpus_fr]
print (inputs[0])

[514, 340, 63, 375, 367, 370, 140, 4, 1, 21, 156, 108, 515]


We are removing the sentence with huge size because - 


*   If we have a huge dataset then the time taken to process it will be also more. Hence for the demo purpose we are removing the sentences which have length greater than 20 words
*   Also we are going to pad the sentences and if we consider the length of the longest sequence then their might be more sparse matrices.. 



In [0]:
MAX_LENGTH = 20
idx_to_remove = [count for count, sent in enumerate(inputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]
idx_to_remove = [count for count, sent in enumerate(outputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]